In [137]:
import pandas as pd

# Load the CSV file
training_data = pd.read_csv('New_News_Train.csv')
testing_data = pd.read_csv("New_News_Test.csv")

# Separate the text and labels
texts = training_data['Removed_stopwords'].tolist()
labels = training_data['Type'].tolist()

In [139]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

# Tokenize the text data
encodings = tokenizer(texts, truncation=True, padding=True, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [140]:
from transformers import AutoModel

# Load the pre-trained model
model = AutoModel.from_pretrained("aubmindlab/bert-base-arabertv2")

# Get the word embeddings
outputs = model(**encodings)
word_embeddings = outputs.last_hidden_state

# Aggregate the word embeddings (e.g., mean)
text_embeddings = word_embeddings.mean(dim=1)

: 

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode the labels
encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Train a LogisticRegression classifier
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

In [ ]:
# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Calculate the accuracy score
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Testing embedding on one sample sentence

In [124]:
# with torch.no_grad():
#     embeddings_model_output = mbeddings_emodel(**encoded)
#     embeddings = embeddings_model_output.last_hidden_state

# print(f'Shape: {embeddings.shape}')
# print(embeddings)

# model = nn.Transformer(
#     d_model=768,  # input and output dimension
#     nhead=8,  # number of heads in multi-head attention
#     num_encoder_layers=6,  # number of encoder layers
#     num_decoder_layers=6,  # number of decoder layers
#     dim_feedforward=512,  # dimension of feedforward network
#     dropout=0.1  # dropout probability
# )
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads, dropout):
        super(TransformerClassifier, self).__init__()
        self.transformer = nn.Transformer(
            d_model=input_dim,
            nhead=num_heads,
            dropout=dropout
        )
        self.classification_head = nn.Linear(input_dim, num_classes)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, src):
        output = self.transformer(src, src)
        output = self.classification_head(output)
        output = self.softmax(output)
        # output = torch.argmax(output)
        return output

In [125]:
model = TransformerClassifier(input_dim=768, num_classes=4, num_heads=8, dropout=0.1)
print(model)

/Users/colin/dev/upwork/arabic_embedding/.venv/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TransformerClassifier(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (linear1): Linear(in_features=768, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=768, bias=True)
          (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (self_attn): MultiheadA

Setting up for using full dataset

In [118]:
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

In [119]:
class AraBERTDataset(Dataset):
    def __init__(self, file, tokenizer, model):
        self.df = pd.read_csv(file)
        self.data = self.df['Removed_stopwords'].values
        self.labels = self.df['Type'].values
        self.tokenizer = tokenizer
        self.model = model

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = self.data[index]
        label = self.labels[index]
        label = F.one_hot(torch.tensor(label), num_classes=4)
        encoded = self.tokenizer.encode_plus(text, return_tensors='pt', padding='max_length', truncation=True)
        with torch.no_grad():
            model_output = self.model(**encoded)
            embeddings = model_output.last_hidden_state[:, 0, :]
        # inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        # outputs = model(**inputs)
        # embeddings = outputs.last_hidden_state[:, 0, :]
        return embeddings, label
        # return {
        #     'input_ids': encoded['input_ids'].flatten(),
        #     'attention_mask': encoded['attention_mask'].flatten()
        # }

In [120]:
training_data = AraBERTDataset('New_News_Train.csv', tokenizer, embedding_model)
testing_data = AraBERTDataset('New_News_Test.csv', tokenizer, embedding_model)

In [121]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
test_dataloader = DataLoader(testing_data, batch_size=8, shuffle=False)

In [136]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

EPOCHS = 5
for epoch in range(EPOCHS):  # Fine-tune for 5 epochs
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        inputs, labels = batch
        # print(inputs.shape, labels.shape)
        inputs = inputs.to(device)
        inputs = torch.squeeze(inputs)
        # print(inputs.shape, labels.shape)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        # print(outputs.dtype)
        # print(labels.dtype)
        labels = labels.float()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}')
    model.eval()

KeyboardInterrupt: 